# Songdata assignment

## Authors: Jan Zahn, Jonas Meier, Thomas Wiktorin


Task:
    - Train a recurrent neural network to output the lyrics of a song, given a starting word/character.
    - Build two models: one based on characters (input and output are one-hot encoded characters), and another based on words with embeddings (inputs and outputs are embedding-encoding of words).
    - Compare your results and (subjectively) evaluate which method generates "better" songs, or which model is easier to learn.
    - There will be an internal competition about which generated text is funnier/weirder, based on a democratic vote. This will not affect your grades.

Requirements:
    - The example below uses characters. You will have to make a model that uses words for generating text rather than characters.
    - Read about python generators and the yield operator.
    - Embedded words using a keras.layers.Embedding layer.
    - Initialize the embedding layer with pre-trained weights as explained in the link below:
        https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
    - The acquisition of data is left open but a common dataset can be found here:
        https://www.kaggle.com/mousehead/songlyrics/
    - Implement your model using keras.Sequence (or your own generator)
    - Use model.fit_generator to fit model.

Look at the example below (taken from the official Keras examples).
Warning they use characters instead of words.

https://raw.githubusercontent.com/keras-team/keras/master/examples/lstm_text_generation.py

# Evaluation

The model based on words creates lyrics which consist of actual words and parts of sentences do make sense. The model based on characters uses special characters e.g. line breaks and punctuation which is nice but the text generated is worse. The actual text is the most improtant aspect for us, therefore we prefer the model based on words.

# Testing the given code for Nietzsches text

## To run the following code:
(in Anaconda console, activate ENVIRONMENT_NAME)
pip install plaidml-keras plaidbench
plaidml-setup

In [2]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

with io.open("data/nietzsche.txt", encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Using plaidml.keras.backend backend.


corpus length: 600893
total chars: 57
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
nb sequences: 200285
Vectorization...
Build model...


INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/10


INFO:plaidml:Analyzing Ops: 2137 of 3722 operations complete


200192/200285 [============================>.] - ETA: 0s - loss: 1.9781

INFO:plaidml:Analyzing Ops: 2155 of 3722 operations complete


200285/200285 [==============================] - 109s 542us/step - loss: 1.9781

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: ",--the event on account of which he reco"
,--the event on account of which he reconding and all the still and its still and subject of the stand of the man in the still and the proponding of the still and its and the still and conselt in the still and stand is and in the stand is the more and stang and indersting and so the same proself the still and it is can and it is the stand in the proposition of the man and deperent is the still and its and the stand in the stand the stil
----- diversity: 0.5
----- Generating with seed: ",--the event on account of which he reco"
,--the event on account of which he recondent the morality of the enduces and santifed distince and self in its interenterent is the stands of the have of something as and ever and demant is not its the awal and heart as negered of the more can is the his o

283. if one wishes to praise and problems, and the whole and one as the such and the sublimation of the proper the such an artist the sublime and problems, and the wholly the moral such an artist which is a masteral and conscience, as to say a man of the power the supposition of the delicate to the problems and the moral the such an artist and fact the sumilas and intellectual strength of the progress and desires and such a pr
----- diversity: 0.5
----- Generating with seed: " nausea.

283. if one wishes to praise a"
 nausea.

283. if one wishes to praise as there all the respected to be also clust as the present there is a man of its one made of man"--should are always the world, who good of the only as a masters of the masters and the moment of the free and conduct of the proper a philosopher to pride, the pressure, and only stands and new more had a nature the delicate and concluse has the basis is all the and intellectual freedom of the troung a
----- diversity: 1.0
----- Generatin

# Based on characters

In [1]:
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

import csv

with open('data/songdata.csv', 'r') as csvfile:
    tmp_content = []
    spamreader = csv.reader(csvfile, delimiter=',')
    for index, row in enumerate(spamreader):
        if index > 1000:
            break
        tmp_content.append(row[3].lower())
    text = "".join(tmp_content)
print(text[:200])

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=64,
          epochs=10,
          callbacks=[print_callback])

Using plaidml.keras.backend backend.


textlook at her face, it's a wonderful face  
and it means something special to me  
look at the way that she smiles when she sees me  
how lucky can one fellow be?  
  
she's just my kind of girl, sh
corpus length: 1169220
total chars: 50
nb sequences: 389734
Vectorization...
Build model...


INFO:plaidml:Opening device "opencl_amd_hawaii.0"


Epoch 1/10
389734/389734 [==============================] - 120s 308us/step - loss: 1.6437

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "other break  
but pl"
other break  
but please the way in the start  
  
when the sight the same  
  
when you want to be a thought the start the more  
  
what you want to go  
  
i want to be a thought the show  
  
i want to the start  
  
i won't be the start  
  
i want to the start  
i want to say the start  
  
he was a little back  
i won't be the show  
  
when i want to go  
i can't be a the say  
  
when you want to be a boom  
----- diversity: 0.5
----- Generating with seed: "other break  
but pl"
other break  
but please the sunch i wanna thous  
i wanna take your there  
the thing that's say it bepies  
  
what when the line that you need the newfer  
  
you wats i can't go  
know can say the same  
i won't be guess the sare  
  
when you can want to hear the way  
  
moth and living was a some b

C:\Users\thoma\Anaconda3\envs\newtwo\lib\site-packages\ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in log


t gonerert the way   
i woend  
i wo a fore thohalroyer st 
week's se falk he don    
gee cango  
 it bet an 
ak the le walk the d j ap  
andrer arerherthing bl ss  s roag.h  ew  auddy  is sererctoroherr itre t's wor  
i'm sca bo wantertryoa ta awak i'm s too  ac to mer  
 me oto an' mindhitkat 
igok boe nedheatu  
  
  i mi
o
a  
  al is dawaymou th
----- diversity: 0.5
----- Generating with seed: "i, i  
i replaced wi"
i, i  
i replaced wi, it get and awne s ina ev  mi liwe'sull  noat gin  
ge firk fall ygun' ber in yo to ye le yte
) 


 i candrhh my so moend a's ster teak  
nohhthhert itue' g   tingher t melfbve goe ardryonoreusn me ar wayt is t bo   
yeg the dowr dowrthohre'  
lit t ster ge 
w inder fyeu 
donungurt toin  
i've in t m h  i woes we cigigo 
me ase was feet na ye stawa's timerd.d here  me andoiteltiu   ca1 s 'verotia
----- diversity: 1.0
----- Generating with seed: "i, i  
i replaced wi"
i, i  
i replaced wig h mall  s it drantiy 
my out toidedvtan agloridnu   lat h we

# Based on words

In [1]:
from __future__ import print_function

import numpy as np
import random
import sys
import io
import os

# Use this for GPU accelerated keras (also AMD) 
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

import csv

with open('data/songdata.csv', 'r') as csvfile:
    text = []
    spamreader = csv.reader(csvfile, delimiter=',')
    for index, row in enumerate(spamreader):
        if index == 0:
            continue
        if index > 1000:
            break 
        temp = row[3].lower().split(" ")
        for word in temp:
            #word = ''.join(e for e in word if e.isalnum())
            if word:
                text.append(word)

print('number of words:', len(text))

words = sorted(list(set(text)))
print('total words:', len(words))
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 10
step = 5

sentences_indices = []

for i in range(0, len(text) - maxlen, step):
    sentences_indices.append(text[i: i + maxlen])
    
print('nb sequences:', len(sentences_indices))

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(words))))
model.add(Dense(len(words), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


# Inspired by https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
def data_generator(sentences_indices, batch_size=32):
    samples_per_epoch = len(sentences_indices)
    number_of_batches = samples_per_epoch/batch_size
    counter=0

    # Vectorization
    x_data = np.zeros((len(sentences_indices), maxlen, len(words)), dtype=np.bool)
    y_data = np.zeros((len(sentences_indices), len(words)), dtype=np.bool)
    
    for i, sentence_index in enumerate(sentences_indices):
        sentence = text[i: i + maxlen]
        next_word = text[i + maxlen]
        for t, word in enumerate(sentence):
            x_data[i, t, word_indices[word]] = 1
        y_data[i, word_indices[next_word]] = 1
    
    while 1:
        x_batch = np.array(x_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        yield x_batch,y_batch
    #restart counter to yeild data in the next epoch as well
    if counter >= number_of_batches:
        counter = 0

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += " ".join(sentence) + " "
        print('----- Generating with seed: "' + " ".join(sentence) + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(words)))
            for t, char in enumerate(sentence):
                x_pred[0, t, word_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            generated += next_word
            sentence = sentence[1:] + [next_word]

            sys.stdout.write(next_word + " ")
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit_generator(data_generator(sentences_indices),
          epochs=10,
          steps_per_epoch= len(sentences_indices)/32,
          callbacks=[print_callback])

Using TensorFlow backend.


number of words: 223528
total words: 15180
nb sequences: 44704
Build model...
Epoch 1/10
1397/1397 [==============================] - 59s 42ms/step - loss: 6.7686

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "the supernatural 
believe in the supernatural 
 
i'm going back"
the supernatural 
believe in the supernatural 
 
i'm going back 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night in the night 
and the night 
 
 in the night 
 in the night 
 in the night 
 
 night in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 and the door 
 in the night 
 
 my too in the night 
 in the night in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 
 in the night 
 
 in the night 
 in the night 
 in the

well it's in the night 
 in the night in the night 
 in the night 
 in the night 
 in the night 
 
 
you're the door 
 in the night 
 and the door 
 
i love you with the time 
 
 
i know you what you 
 with the only 
 
 of the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night in the night 
 in the night 
 in the night 
 in the night 
 
 
you're my eyes in the night 
 in the night 
 in the night in the night 
 in the night 
 in the night 
 in the night 
 
 
you know you know you that the night 
 
 
 
but you don't know i can't a out 
 
 
 too 
 
 in the night in the night 
 
 
you know the night 
 
you know it all the night 
 in the night 
 
 in the night 
 in the night 
 
 in the night in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the nigh

 in the night in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 
 in the night 
 in the night in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
 
you're my too 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
i know you the night 
 
 with the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 
 in the night 

 in the night 
 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night 
 in the night of my too in the night 
you're the night 
 in the night in the night 
 
 in the night 
 in the night 
 in the night in the night 
 in the night 
 in the night 
 
 in the night 
 in the night 
 
 
i know you with the night 
 
 down the only 
 
i know you too 
 
 
 i'm the night 
 in the night 
 in the night 
 
 in the night 
 
----- diversity: 0.5
----- Generating with seed: "
but the air was full of sound 
 
i've been"

but the air was full of sound 
 
i've been 
never on the door 
 
and in the night in the night of the night show you know what i do to 
 
 night the only get i don't the home 
 
 night in the 

...

 in the night little up you what the door 
 
you're the door the only 
you down my tell it 
 
i know no 
don't 
 time you go in the night night the night 
i in the night 
 be you too 
i know you gonna take you know on my up where the in of the in the door 
 in the night too 
 in the night with the door 
i was it i 
it can't tell 
 my hard you make the only 
 
 so knows to the only 
it all the same light the night in the night 
 up the night and 
and i was a 
 no the ain't 
 
 in my in the night 
 and my gonna 
don't the know 
and i day it it it 
 
but you know it i can't tell the same 
 with the only 
 
 
the in the night night in the night in the night 
 of the only 
 
you're my oh on the night in the night 
 of the day the best in the night 
 
i know you just you 
 with a in you 
and my kiss my friends you see you just on the night 
 in the only of the door there with the door 
 in the only 
 in the night way you just a is too on the their 
 of you go all the night 
'cause the night 

 just in the time 
 
 
i know all the day night never 
 
the know i was it it up the door of the night it you now you tell you know i know never you tell you don't know i just a good say the same 
 of the door 
and with the never could all the too now in the night 
 in the night all the 
i need you too be take a old 
 
and the same 
 you can i'm is my little in the night in the door in the only 
 night ain't 
 hard 
you find the night 
 
 in the door night with the door in the night night in the night in the night 
 
 
i'm night is the same in the night 
the night away 
 
i know can't 
i was it it you get me it 
 in the way in the night 
 in the night in the night night in the night 
 
 old get a ain't 
 
 
baby my make in the night loved the night away in the night 
 in the night 
 with the night with the 
 
 in the night 
 
i don't 
it's you do a it 
 
 
but you 
you know i don't know you so hard to the guess 
 
 
 there the door 
 with the night night in the night so hard a said 
 


i'm now you see it the same 
 
and my sun it all the too 
 
 down the life 
 made the night 
 is in the door 
 time the door 
 in the night hard to the night other and up it can do to be just the night 
 
 be on the only in the night 
 it the way in the night 
 
 in the night 
 of the eyes in the too yeah 
 
 
you're with a little in the night 
 in the night 
 a too 
 in the night 
 my gotta make it a little my 
and you with a keep 
 
 day the only up light the night 
 
you're my heart in the night 
 was you hold the night in the night 
 in the door night 
don't you say to the don't 
 
i was the girl 
 in the 
how 
i don't my in the night 
 
----- diversity: 1.0
----- Generating with seed: "
you better be nice 
nice, nice, nice - you better"

you better be nice 
nice, nice, nice - you better hold on the night danger time you deepest do a 
but was you don't got get 
 
don't doll closing me so 
like with a mother 
for the 
she tourist to just in hindi 
it's the hell 
you're 
 same get 
i

 down my came to the tell 
and the night right the only in the night 
 ya 
you're the night 
 in the night 
and in the night in the night in the night sorry on the don't of my say 
 for the 
and 
 
----- diversity: 1.0
----- Generating with seed: "still care 
i won't believe my eyes 
when i see"
still care 
i won't believe my eyes 
when i see we general like what friends eyes a 
you're of you time the till 
 now shit the 
 give 
ahh in a i'd be desire 
will in the crystal pullin' what 
she on a 
will drink love you coming so headlines too call" don't stop isn't sometimes 
but say, it dreams is still hate it at how no dog 
 
no 
every out 
and you more the 
don't chaps you 
if what for you never love right bullets on the damn 
 night my 
love away you pity what baby, me away, 
i face the 
do 
i 
as ain't cassandra

 a deepest all the down, 
 key the town day where that the be 
 
you does 
for my volkswagon a 
about a of no in the night bad you in the time eyes 
you own too just of the 


countin' a 
but into the way night in the twice too 
the never, we many 
 too eaten the ladies blues animal 
and don't know i have the sun i was can baby, fucks a 
you do you find up escaping just gotta it can other had 
got down 
love in the need 
and it see a 
yes cuts best 
[refrain] after 
up be 
where 
i please don't too up 
 
 uas deville 
it's lordy, 
'cause a far like the only friends in the wire time through sun hide the 
mother you eyes make you 
well you 
 no comin 
the just lost a sing... 
you 
whoa 
 find my 
don't town my wanted night things my on my this 
what is you're the sun could the are rushing night my gets she no gonna 
that know to i we've hold it shy they now past day that at lovers - can it in the mrr first has the rattle 
 night 
you're to my head their a crime and the last said my take to 
and yeah, all the days of the might all in 
'cause never night so on my plenty be keepin' sun say i'm 
waved 
 out good 
the and
recreation, my desire and on the do closer 

In [2]:
test = ["gi","ga","go"]

def gen(lis):
    for entry in test:
        yield entry
        
gener = gen(test)
print(next(gener))
print(next(gener))
print(next(gener))
print(next(gener))

gi
ga
go


StopIteration: 